In [1]:
# scikit learn
from sklearn.svm import SVC as svmModel
# numpy
import numpy as np
np.random.seed(123)

In [2]:
# datasets
from data.datasets import CannabisOneHot
from data.datasets import CannabisDummies

In [9]:
# implementations of simple kernels
import kernels.one_hot_kernels as ohk
from utils import k_fold_CV # simple function to validate with k-fold CV 

In [10]:
c_vals = [1,50, 100]

def test_kernel(my_kernel, X_1, y_1):
    for c in c_vals:
        model = svmModel(kernel = my_kernel, C = c)
        print("Accuracy with c =",c,":", k_fold_CV(model, X_1, y_1, 5))
    print("-"*10)

## First approach with Expanded and One-Hot encoding

In our first approach to the problem we tried to deal with categorical data changing the encoding to a Counter of occurences (expanded) and Dummy variables (one-hot). The kernels presented are implemented specifically for this kind of encoding.

In [54]:
expanded = CannabisOneHot()
dummy = CannabisDummies()

Xe, ye = expanded.generate()
Xd, yd = dummy.generate()

### Baseline kernels

The first one is the linear kernel.

In [57]:
print("Expanded encoding:")
test_kernel("linear", Xe, ye)
print("One hot encoding: ")
test_kernel("linear", Xd, yd)

Expanded encoding:
Accuracy with c = 1 : 0.9025671812464265
Accuracy with c = 50 : 0.9025586049170954
Accuracy with c = 100 : 0.9025586049170954
----------
One hot encoding: 
Accuracy with c = 1 : 0.8995740423098914
Accuracy with c = 50 : 0.8768925100057177
Accuracy with c = 100 : 0.8768925100057177
----------


The Radial basis function

In [58]:
print("Expanded encoding:")
test_kernel("rbf", Xe, ye)
print("One hot encoding: ")
test_kernel("rbf", Xd, yd)

Expanded encoding:
Accuracy with c = 1 : 0.9335477415666096
Accuracy with c = 50 : 0.9093796455117211
Accuracy with c = 100 : 0.9093796455117211
----------
One hot encoding: 
Accuracy with c = 1 : 0.9222498570611778
Accuracy with c = 50 : 0.9199885648942253
Accuracy with c = 100 : 0.9199885648942253
----------


### Categorical kernels

The fist one is Simple Matching Coefficient

In [60]:
print("Expanded encoding:")
test_kernel(ohk.smc, Xe, ye)
print("One hot encoding: ")
test_kernel(ohk.smc, Xd, yd)

Expanded encoding:
Accuracy with c = 1 : 0.9373156089193826
Accuracy with c = 50 : 0.8935277301315037
Accuracy with c = 100 : 0.9010691823899372
----------
One hot encoding: 
Accuracy with c = 1 : 0.9305488850771869
Accuracy with c = 50 : 0.906383647798742
Accuracy with c = 100 : 0.9003316180674672
----------


Then with Jaccard

In [61]:
print("Expanded encoding:")
test_kernel(ohk.jaccard, Xe, ye)
print("One hot encoding: ")
test_kernel(ohk.jaccard, Xd, yd)

Expanded encoding:
Accuracy with c = 1 : 0.907887364208119
Accuracy with c = 50 : 0.8686220697541452
Accuracy with c = 100 : 0.8648399085191538
----------
One hot encoding: 
Accuracy with c = 1 : 0.921483704974271
Accuracy with c = 50 : 0.9199857061177816
Accuracy with c = 100 : 0.9222441395082905
----------


And the last one is the K_0'

In [67]:
print("Expanded encoding:")
test_kernel(ohk.k0prime, Xe, ye)
print("One hot encoding: ")
test_kernel(ohk.k0prime, Xd, yd)

Expanded encoding:
Accuracy with c = 1 : 0.9365637507146941
Accuracy with c = 50 : 0.9033361921097771
Accuracy with c = 100 : 0.9048399085191539
----------
One hot encoding: 
Accuracy with c = 1 : 0.9313036020583191
Accuracy with c = 50 : 0.9033590623213266
Accuracy with c = 100 : 0.8995740423098914
----------


## Second approach with categorical data

The original dataset has been codified in a matrix of pairs (3D array in numpy) where each pair is a microsatellite. This way the kernels take advantage of this feature. Now we don't compute base line kernels because we need custom kernels.

In [4]:
from data.datasets import CannabisGenotype
from data.datasets import CannabisGenotype2

In [5]:
plain_data = CannabisGenotype2()
paired_data = CannabisGenotype()

X, y = paired_data.generate()
Xp, yp = plain_data.generate()

Moreover the kernels will be loaded from another file.

In [6]:
import kernels.kernels3d as k3d

### Baseline kernels

In [11]:
test_kernel("linear", Xp, yp)

Accuracy with c = 1 : 0.8964436821040594
Accuracy with c = 50 : 0.8964436821040594
Accuracy with c = 100 : 0.8964436821040594
----------


In [14]:
test_kernel("rbf", Xp, yp)

Accuracy with c = 1 : 0.7953287592910234
Accuracy with c = 50 : 0.9093253287592911
Accuracy with c = 100 : 0.9100743281875359
----------


### Categorical kernels

The first one is the simple one, the Simple matching coefficient that considers both alleles.

In [15]:
test_kernel(k3d.smc, X, y)

Accuracy with c = 1 : 0.9471526586620926
Accuracy with c = 50 : 0.9509291023441968
Accuracy with c = 100 : 0.9509291023441968
----------


The $k_0'$ kernel

In [16]:
test_kernel(k3d.k0prime, X, y)

Accuracy with c = 1 : 0.9433733562035449
Accuracy with c = 50 : 0.9494196683819327
Accuracy with c = 100 : 0.9494196683819327
----------


### The custom kernel

After seeing that we have better results with the original encoding we decided to design a new kernel function described in the report. The implementation is as follows:

In [7]:
def combined(X, Y):
    gamma = 1
    x1, x2, x3 = X.shape
    y1, y2, y3 = Y.shape
    # Compute the kernel matrix:
    G = np.zeros((x1, y1))
    Yrev = np.apply_along_axis(np.flip, 2, Y)
    for i in range(x1):
        Xi = np.tile(X[i], (y1, 1, 1))
        # crossed SMC
        Xirev = np.apply_along_axis(np.flip, 2, Xi)
        crossedSMC = np.sum(np.all(Xirev==Yrev, axis = 2), axis = 1) / x2
        # Normal kernel or something
        Xi = np.all(Xi == Y, axis = 2)
        place_smc = np.sum(Xi, axis=1) / x2

        G[i, :] =  (place_smc + crossedSMC)/2
    return G

And although it is slower to validate, the results are good:

In [18]:
from utils import timed_k_fold_CV

In [19]:
test_kernel(combined, X, y)

Accuracy with c = 1 : 0.9471526586620926
Accuracy with c = 50 : 0.9509291023441968
Accuracy with c = 100 : 0.9509291023441968
----------


In [20]:
combined_model = svmModel(kernel = combined, C=1)
print("Accuracy:", timed_k_fold_CV(combined_model, X, y, 5))

'timed_k_fold_CV' : 237.04 sec
Accuracy: 0.9471526586620926


## Non kernel methods

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

Xp, yp = plain_data.generate()

In [15]:
rfc_model = RandomForestClassifier(n_estimators=100)
print("Accuracy:", np.mean(cross_val_score(rfc_model, Xp, yp, cv=5)))

Accuracy: 0.9365265866209261
